In [8]:
!pip install QuantLib

In [9]:
import pandas as pd
import QuantLib as ql

In [10]:
pgbs = pd.DataFrame(
    {'maturity': ['15-06-2020', '15-04-2021', '17-10-2022', '25-10-2023',
                  '15-02-2024', '15-10-2025', '21-07-2026', '14-04-2027',
                  '17-10-2028', '15-06-2029', '15-02-2030', '18-04-2034',
                  '15-04-2037', '15-02-2045'],
    'coupon': [4.8, 3.85, 2.2, 4.95,  5.65, 2.875, 2.875, 4.125,
                2.125, 1.95, 3.875, 2.25, 4.1, 4.1],
    'px': [102.532, 105.839, 107.247, 119.824, 124.005, 116.215, 117.708,
            128.027, 115.301, 114.261, 133.621, 119.879, 149.427, 159.177]})

calendar = ql.TARGET()
today = calendar.adjust(ql.Date(19, 12, 2019))
ql.Settings.instance().evaluationDate = today

bondSettlementDays = 2
bondSettlementDate = calendar.advance(
    today,
    ql.Period(bondSettlementDays, ql.Days))
frequency = ql.Annual
dc = ql.ActualActual(ql.ActualActual.ISMA)
accrualConvention = ql.ModifiedFollowing
convention = ql.ModifiedFollowing
redemption = 100.0

instruments = []
for idx, row in pgbs.iterrows():
    maturity = ql.Date(row.maturity, '%d-%m-%Y')
    schedule = ql.Schedule(
        bondSettlementDate,
        maturity,
        ql.Period(frequency),
        calendar,
        accrualConvention,
        accrualConvention,
        ql.DateGeneration.Backward,
        False)
    helper = ql.FixedRateBondHelper(
            ql.QuoteHandle(ql.SimpleQuote(row.px)),
            bondSettlementDays,
            100.0,
            schedule,
            [row.coupon / 100],
            dc,
            convention,
            redemption)

    instruments.append(helper)

params = [bondSettlementDate, instruments, dc]

piecewiseMethods = {
    "PiecewiseFlatForward": ql.PiecewiseFlatForward(*params),
    "LogLinearDiscount": ql.PiecewiseLogLinearDiscount(*params),
    "LogCubicDiscount": ql.PiecewiseLogCubicDiscount(*params),
    "LinearZero": ql.PiecewiseLinearZero(*params),
    "CubicZero": ql.PiecewiseCubicZero(*params),
    "LinearForward": ql.PiecewiseLinearForward(*params),
    "SplineCubicDiscount": ql.PiecewiseSplineCubicDiscount(*params)
}




In [11]:
df = pd.DataFrame(index=[row[0] for row in piecewiseMethods['LogLinearDiscount'].nodes()])
for curve in piecewiseMethods:
    dfs = [piecewiseMethods[curve].discount(idx) for idx in df.index]
    df[curve] = dfs

display(df)

                     PiecewiseFlatForward  LogLinearDiscount  \
December 23rd, 2019              1.000000           1.000000   
June 15th, 2020                  1.002192           1.002192   
April 15th, 2021                 1.007589           1.007589   
October 17th, 2022               1.009800           1.009800   
October 25th, 2023               1.006457           1.006457   
February 15th, 2024              1.003781           1.003781   
October 15th, 2025               0.994422           0.994422   
July 21st, 2026                  0.987640           0.987640   
April 14th, 2027                 0.978990           0.978990   
October 17th, 2028               0.966528           0.966528   
June 15th, 2029                  0.959323           0.959323   
February 15th, 2030              0.947371           0.947371   
April 18th, 2034                 0.887530           0.887530   
April 15th, 2037                 0.823855           0.823855   
February 15th, 2045              0.68268